## Pubsub GCP
***

O Pub/Sub do Google Cloud Platform (GCP) é um serviço de mensagens assíncrono e altamente escalável que permite que aplicativos troquem informações de forma confiável e em tempo real. Pense nele como um carteiro super eficiente que garante que as mensagens enviadas por um aplicativo (o "**publisher**") cheguem a todos os aplicativos interessados (os "**subscribers**"), mesmo que eles estejam rodando em momentos diferentes ou em grande escala. Ele desacopla os sistemas, tornando-os mais resilientes e flexíveis.

1. Se autentique no GCP
2. Vai na aba Pubsub
3. Clique em Topics
4. Clique em Create Topic
5. Insira o nome do topic e clica em Create
6. Copia a URL no formato: projects/{project_id}/subscriptions/{subscription_id}
7. Verifique se sua SA tem permissão para publicar no topico e consumir a subscription criada.
8. Execute no terminal o comando: export 

### Consumindo fila pubsub
***

In [ ]:
from google.pubsub_v1 import SubscriberAsyncClient, PullRequest, ReceivedMessage, PubsubMessage
from google.api_core.exceptions import GoogleAPIError
from google.api_core.retry_async import AsyncRetry
from asyncio import Event, Queue
import traceback
import json
import time
import asyncio
import random
import os
import signal
import logging

In [ ]:
retry = AsyncRetry()

In [ ]:
PUBSUB_ENABLED = 1
MAX_MESSAGES = 30
PROCESS_TIME_SLEEP = 0.5
MAX_CONCURRENT = 50
EVALUATE_MESSAGE_TIMEOUT = 60
PROJECT_ID = "vksoftware"
SUBSCRIPTION_ID = "notebook-sub"
QUEUE = f"projects/{PROJECT_ID}/subscriptions/{SUBSCRIPTION_ID}"

In [ ]:
def force_shutdown():
    """
    Força o shutdown do serviço.
    """

    os.kill(os.getpid(), signal.SIGINT)

In [ ]:
def shutdown(stop_event: Event, sig: signal.Signals) -> None:
    """
    Setando o evento de parada.
    """

    print(f"Sinal de sair recebido: {sig.name}")
    stop_event.set()
    print("Evento de parada setado.")

In [ ]:
def setup_signal_handler(stop_event: Event) -> None:
    """
    Observador de sinais para parada
    do script ou graceful shutdown.
    """

    loop = asyncio.get_running_loop()

    # kill -l (mostra todos os comandos de sinais)
    # SIGKILL -> Mata o processo forçado e não tem oq fazer (kill -9)
    # SIGHUP -> Recebeu um evento de desligamento do processo (desescalonamento de um pod)
    # SIGTERM -> Recebeu um evento para terminar o processo (parada de um pod)
    # SIGINT -> Recebeu um evento de interrupção (Ctrl+C)
    # SIGALRM -> Dispara um alarme e finaliza de forma controlada
    # Se ocorrer qualquer um desses sinais execute o evento de parada (shutdown)
    for sig in (signal.SIGHUP, signal.SIGTERM, signal.SIGINT, signal.SIGALRM):
        loop.add_signal_handler(sig, shutdown, stop_event, sig)

In [ ]:
async def run_something(payload: dict, trace_id: str) -> bool:
    """
    Roda algo do tipo IO Bound como queries no BD ou requisições http.
    """
    
    try:
        print("Iniciou a execução.")
        await asyncio.sleep(random.choice([1,2,3,4,5]))
        print("Finalizou a execução.")
    except Exception as e:
        return False

    return True

In [ ]:
async def process_message(received_message: ReceivedMessage, subscription: SubscriberAsyncClient) -> None:
    """
    Realiza o processamento das mensagens
    """

    message: PubsubMessage = received_message.message
    payload: dict = json.loads(message.data)

    success = await run_something(payload, trace_id)

    if success:
        await subscription.acknowledge(
            subscription=QUEUE,
            ack_ids=[received_message.ack_id],
            retry=retry
        )
    else:
        await subscription.modify_ack_deadline(
            subscription=QUEUE,
            ack_ids=[received_message.ack_id],
            ack_deadline_seconds=0,
            retry=retry
        )

In [ ]:
async def get_messages_from_pubsub_and_put_in_queue(message_queue: Queue, subscription: SubscriberAsyncClient):
    """
    Pega as mensagens do pubsub e insere dentro da queue
    """

    messages_to_get = MAX_MESSAGES - message_queue.qsize()
    print(f"Pegando {messages_to_get} mensagens do pubsub.")

    if messages_to_get <= 0:
        return

    try:
        print("Puxando as mensagens do pubsub.")
        response = await asyncio.wait_for(
            subscription.pull(PullRequest(
                subscription=QUEUE,
                max_messages=messages_to_get,
                return_immediately=True
            ), retry=retry),
            timeout=30)
    except asyncio.TimeoutError as error:
        print("PUBSUB PULL TIMEOUT. Finalizando o serviço de busca de mensagens.")
        force_shutdown()
        return
    except GoogleAPIError as error:
        print('QUEUE BLOKED. Finalizando o serviço de busca de mensagens.')
        force_shutdown()
        return
    except Exception as error:
        print('QUEUE DEAD. Finalizando o serviço de busca de mensagens.')
        force_shutdown()
        return

    if len(response.received_messages) == 0:
        print("PUBSUB EMPTY")
        return

    print(f"Recebido {len(response.received_messages)} mensagens do pubsub.")
    for message in response.received_messages:
        print("Colocando as mensagens recebidas na fila interna.")
        await message_queue.put(message)

In [ ]:
async def producer(queue: Queue, stop_event_consumer: Event, stop_event_producer: Event, subscription: SubscriberAsyncClient) -> None:
    """
    Recepção assíncrona de mensagens de uma fila PubSub do GCP.
    """

    while True:
        if stop_event_consumer.is_set():
            stop_event_producer.set()
            print("STOP EVENT SET. Finalizando o serviço de busca de mensagens.")
            return

        with open('.timestamp.healthcheck', 'w') as f:
            f.write(str(time.time()))

        print("Pegando mensagen do pubsub")
        await get_messages_from_pubsub_and_put_in_queue(queue, subscription)
        await asyncio.sleep(1)

In [ ]:
async def consumer(queue: Queue, stop_event_producer: Event, subscription: SubscriberAsyncClient) -> None:
    """
    Consome as mensagens da fila e as processa usando asyncio.create_task
    em conjunto com asyncio.wait (que se comporta de forma similar
    ao asyncio.as_completed) para limitar o número de mensagens processadas
    simultaneamente.
    """

    pending_tasks = set()

    while True:
        if stop_event_producer.is_set() and queue.empty():
            print(f"STOP EVENT SET. Finalizando o serviço de monitoramento de mensagens com {queue.qsize()} msg na fila.")
            break

        try:
            received_message: ReceivedMessage = queue.get_nowait()
            print(f"Peguei a mensagem: {received_message.ack_id}")
        except asyncio.QueueEmpty:
            print("QUEUE EMPTY")
            await asyncio.sleep(1)
            continue
        except Exception as error:
            traceback.print_exc()
            print('PROCESSING ERROR. Finalizando o serviço de processamento de mensagens.')
            force_shutdown()
            break
        finally:
            qsize = queue.qsize()
            len_pending_tasks = len(pending_tasks)
            if qsize > 0 or len_pending_tasks > 0:
                print(f"Fila atualmente com {qsize} items e {len_pending_tasks} pendentes")

        # Cria uma task para processar a mensagem
        task = asyncio.create_task(process_message(received_message, subscription))
        pending_tasks.add(task)

        # Ao finalizar, remove a task do conjunto de pendentes
        task.add_done_callback(pending_tasks.discard)

        # Se atingiu o limite de tasks concorrentes, aguarda a conclusão
        # de pelo menos uma
        if len(pending_tasks) >= MAX_CONCURRENT:
            print(f"Limite atingido: {len(pending_tasks)} tasks. Aguardando alguma finalizar...")
            # Espera até que ao menos uma task seja concluída
            await asyncio.wait(
                pending_tasks,
                timeout=EVALUATE_MESSAGE_TIMEOUT,
                return_when=asyncio.FIRST_COMPLETED
            )
            # Assim, o conjunto diminui e permite a criação de novas tasks
            print(f"Uma ou mais tasks finalizaram. Pending tasks count agora: {len(pending_tasks)}")

        queue.task_done()
        await asyncio.sleep(PROCESS_TIME_SLEEP)

In [ ]:
async def main(stop_event_consumer: Event, stop_event_producer: Event) -> None:
    """
    Função principal que inicializa a fila e inicia as
    tasks do listener e do consumer.
    """

    if PUBSUB_ENABLED:
        subscription = SubscriberAsyncClient()

        with open('.timestamp.healthcheck', 'w') as f:
            f.write(str(time.time()))

        message_queue = Queue(maxsize=MAX_MESSAGES + 1)
        all_tasks = []
        setup_signal_handler(stop_event_consumer)

        all_tasks.append(asyncio.create_task(producer(message_queue, stop_event_consumer, stop_event_producer, subscription)))

        for _ in range(MAX_MESSAGES):
            all_tasks.append(
                asyncio.create_task(consumer(message_queue, stop_event_producer, subscription))
            )

        await asyncio.gather(*all_tasks)

In [ ]:
stop_event_consumer = asyncio.Event()
stop_event_producer = asyncio.Event()
current_loop = asyncio.get_event_loop()
current_loop.create_task(main(stop_event_consumer, stop_event_producer))

### Enviando mensagens para a fila
***

In [ ]:
from google.pubsub_v1 import PublisherAsyncClient, PublishRequest, PubsubMessage
import json

In [ ]:
PROJECT_ID = "vksoftware"
TOPIC_ID = "notebook"

In [ ]:
retry = AsyncRetry()

In [ ]:
publisher = PublisherAsyncClient()

In [ ]:
json_payload = json.dumps({"success": True}).encode('utf-8')

In [ ]:
topic_path = publisher.topic_path(PROJECT_ID, TOPIC_ID)

In [ ]:
message = PubsubMessage(data=payload)

In [ ]:
request = PublishRequest(topic=topic_path, messages=[message])

In [ ]:
await publisher.publish(request=request, retry=retry)